In [1]:
import os

In [2]:
%pwd

'c:\\Users\\asus\\OneDrive\\Desktop\\Risk_Score_Predication_With_ML_FLOW\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\asus\\OneDrive\\Desktop\\Risk_Score_Predication_With_ML_FLOW'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_dir: Path
    train_data_dir: Path
    test_data_dir: Path    

In [6]:
from RiskScorePrediction.constants import *
from RiskScorePrediction.utils.common import read_yaml,create_directories


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_dir = config.local_data_dir,
            train_data_dir=config.train_data_dir,
            test_data_dir=config.test_data_dir
        )
        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from RiskScorePrediction import logger
from RiskScorePrediction.utils.common import get_size
import pandas as pd
from sklearn.model_selection import train_test_split

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    

    def download_file(self):
        if not os.path.exists(self.config.local_data_dir):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_dir
            )
        else:
            logger.info(f"Downloaded {get_size(Path(self.config.local_data_dir))} bytes")

    def train_test_split(self):
        try:
            df= pd.read_csv("artifacts/data_ingestion/data.csv")
            train_set,test_set = train_test_split(df,test_size=0.2,random_state=42)
            
            train_set.to_csv(self.config.train_data_dir,index=False,header=True)

            test_set.to_csv(self.config.test_data_dir,index=False,header=True)

            logger.info("Train-Test Split Done")
        except Exception as e:
            raise e






In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.train_test_split()

except Exception as e:
    raise e

[2025-01-29 20:02:55,073: INFO: common: yaml file : config\config.yaml loaded  sucessfully]
[2025-01-29 20:02:55,077: INFO: common: yaml file : params.yaml loaded  sucessfully]
[2025-01-29 20:02:55,080: INFO: common: yaml file : schema.yaml loaded  sucessfully]
[2025-01-29 20:02:55,084: INFO: common: created directory at :artifacts]
[2025-01-29 20:02:55,090: INFO: common: created directory at :artifacts/data_ingestion]
[2025-01-29 20:03:04,890: INFO: 1171044442: Train-Test Split Done]
